In [1]:
import pickle

X = pickle.load(open('X.pickle','rb'))
y = pickle.load(open('y.pickle','rb'))
X_t = pickle.load(open('X_t.pickle','rb'))
y_t = pickle.load(open('y_t.pickle','rb'))

X = X/255.0
X_t = X_t/255.0

In [2]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.applications import EfficientNetB0,EfficientNetB2 ,EfficientNetB5
from tensorflow.keras.models import Sequential
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [9]:
IMAGE_SIZE = [48, 48]

# add preprocessing layer to the front of VGG

pretrained_model = EfficientNetB0(input_shape=IMAGE_SIZE + [3], weights=None, include_top=False)

# don't train existing weights
#for layer in pretrained_model.layers:
    #layer.trainable = False
    

In [10]:
# our layers - you can add more if you want
x = Flatten()(pretrained_model.output)
x2 = Dense(128, activation='relu')(x)
x3 = Dropout(0.2)(x2)
prediction = Dense(5, activation='softmax')(x3)

# create a model object
model = Model(inputs=pretrained_model.input, outputs=prediction)

# view the structure of the model
model.summary()

optimizer = Adam(lr = 0.001)

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer=optimizer,
  metrics=['accuracy']
)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 48, 48, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, 48, 48, 3)    7           rescaling_1[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 49, 49, 3)    0           normalization_1[0][0]            
____________________________________________________________________________________________

In [26]:
optimizer = Adam(lr = 0.000001)

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer=optimizer,
  metrics=['accuracy']
)

In [27]:
model.fit(X, y, batch_size=32, epochs=5)

Epoch 1/5
1585/1585 [==============================] - 65s 37ms/step - loss: 0.8934 - accuracy: 0.6484
Epoch 2/5
1585/1585 [==============================] - 59s 37ms/step - loss: 0.8865 - accuracy: 0.6484
Epoch 3/5
1585/1585 [==============================] - 60s 38ms/step - loss: 0.9009 - accuracy: 0.6416
Epoch 4/5
1585/1585 [==============================] - 59s 37ms/step - loss: 0.8956 - accuracy: 0.6481
Epoch 5/5
1585/1585 [==============================] - 59s 37ms/step - loss: 0.8926 - accuracy: 0.6517


In [28]:
val_loss, val_acc = model.evaluate(X_t, y_t)
print(val_loss)
print(val_acc)

390/390 [==============================] - 6s 14ms/step - loss: 1.1193 - accuracy: 0.5617
1.1193294525146484
0.5617383122444153


In [2]:
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import Adam

In [29]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [12]:
# load json and create model
json_file = open('dataset/model_en0.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("dataset/model_en0.h5")
print("Loaded model from disk")

optimizer = Adam(lr = 0.001)

loaded_model.compile(
  loss='categorical_crossentropy',
  optimizer=optimizer,
  metrics=['accuracy']
)

Loaded model from disk


In [4]:
val_loss, val_acc = loaded_model.evaluate(X_t, y_t)
print(val_loss)
print(val_acc)

449/449 [==============================] - 13s 14ms/step - loss: 1.4867 - accuracy: 0.4176
1.479573369026184
0.423307329416275


In [13]:
loaded_model.fit(X, y, batch_size=32, epochs=10)

Epoch 1/10
1846/1846 [==============================] - 75s 32ms/step - loss: 1.4794 - accuracy: 0.4249
Epoch 2/10
1846/1846 [==============================] - 59s 32ms/step - loss: 1.5079 - accuracy: 0.4131
Epoch 3/10
1846/1846 [==============================] - 60s 32ms/step - loss: 1.4433 - accuracy: 0.4396
Epoch 4/10
1846/1846 [==============================] - 60s 32ms/step - loss: 1.4733 - accuracy: 0.4283
Epoch 5/10
1846/1846 [==============================] - 60s 32ms/step - loss: 1.4213 - accuracy: 0.4489
Epoch 6/10
1846/1846 [==============================] - 61s 33ms/step - loss: 1.4129 - accuracy: 0.4504
Epoch 7/10
1846/1846 [==============================] - 60s 33ms/step - loss: 1.3792 - accuracy: 0.4622
Epoch 8/10
1846/1846 [==============================] - 59s 32ms/step - loss: 1.3976 - accuracy: 0.4633
Epoch 9/10
1846/1846 [==============================] - 60s 33ms/step - loss: 1.5150 - accuracy: 0.4079
Epoch 10/10
1846/1846 [==============================] - 59s 32m